In [ ]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize

stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
#Input your PostGres credentials to connect

dbname = ''
username = ''
host = ''
password = ''

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [ ]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 30
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)

In [ ]:
review_sample

In [ ]:
print(review_sample.loc[24, 'review_text'])

Contractions:
https://gist.github.com/J3RN/ed7b420a6ea1d5bd6d06

In [ ]:
common_nonneg_contr = ["could've",
"he'd",
"he'd've",
"he'll",
"he's",
"how'd",
"how'll",
"how's",
"i'd",
"i'd've",
"i'll",
"i'm",
"i've",
"it'd",
"it'd've",
"it'll",
"it's",
"let's",
"ma'am",
"might've",
"must've",
"o'clock",
"'ow's'at",
"she'd",
"she'd've",
"she'll",
"she's",
"should've",
"somebody'd",
"somebody'd've",
"somebody'll",
"somebody's",
"someone'd",
"someone'd've",
"someone'll",
"someone's",
"something'd",
"something'd've",
"something'll",
"something's",
"that'll",
"that's",
"there'd",
"there'd've",
"there're",
"there's",
"they'd",
"they'd've",
"they'll",
"they're",
"they've",
"'twas",
"we'd",
"we'd've",
"we'll",
"we're",
"we've",
"what'll",
"what're",
"what's",
"what've",
"when's",
"where'd",
"where's",
"where've",
"who'd",
"who'd've",
"who'll",
"who're",
"who's",
"who've",
"why'll",
"why're",
"why's",
"would've",
"y'all",
"y'all'll",
"y'all'd've",
"you'd",
"you'd've",
"you'll",
"you're",
"you've"]

In [ ]:
neg_stops1 = ['no',
 'nor',
 'not',
 'don',
 "don't",
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'but',
 "don'",
 "ain't"]

words_to_be_added = ['us']
stops1 = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [ ]:
for x in neg_stops1:
    if x in stops1:
        stops1.remove(x)
        
stops1

In [ ]:
new_stops = stops1 + common_nonneg_contr
list(set(new_stops))

In [ ]:
#Turning this into text, as it's the less efficient code
"""

def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#I think I adjusted this properly to remove digits
#however for phrases that are hyphenated, like chilled-to-the-bone, the code returns the token 'chilledtothebone.'
#This will need to be fixed
def _clean_review(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = text.split()
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review(text):
    tokens = _remove_stops(_clean_review(text))
    return tokens
    
    
    
"""

In [ ]:
stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review2(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [token for token in text.split() if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review2(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens

In [ ]:
#Turning this into text, as it is less efficient than 2

"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review3(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)
    tokens = [wnl.lemmatize(token, pos = get_wordnet_pos(token)) for token in text.split() if token not in stops]
    return tokens

def _process_review3(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
    
"""

In [ ]:
#This code is slow

"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review4(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [wnl.lemmatize(token, pos= get_wordnet_pos(token)) for token in text.split()]
    tokens = [token for token in tokens if token not in stops]
    return tokens

def _process_review4(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
"""

In [ ]:
"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review5(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)
    tokens = text.split()
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    tokens = [token for token in tokens if token not in stops]
    return tokens

def _process_review5(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
"""

In [ ]:
_clean_review2(review_sample.loc[24, 'review_text'])

In [ ]:
def apply_on_column(data):
    data['review_text'] = data['review_text'].apply(lambda x: _clean_review6(x))
    return data

In [ ]:
def map_on_column(data):
    data['review_text'] = data['review_text'].map(lambda x: _clean_review2(x))
    return data

In [ ]:
#Takes about 8 min 8 sec for 50000 (_clean_review2)
# 15 min 24.965 seconds for 100000
start = time.time()
apply_on_column(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
review_sample.loc[27, 'review_text']

In [ ]:
def iterrows_at(data):
    for i, row in data.iterrows():
        data.at[i, 'review_text'] = _clean_review2(data.at[i, 'review_text'])

In [ ]:
#Takes a little longer.. like 7 min 56 seconds for 50000 reviews (_clean_review2)
start = time.time()
iterrows_at(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
#7 min 31.66 seconds
#15 min 50.93 seconds for 100000
start = time.time()
map_on_column(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review6(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)
    tokenizer = nltk.RegexpTokenizer('\w+\'?\w*')
    tokens = [token for token in tokenizer.tokenize(text) if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review6(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens



In [ ]:
test = "testing hello-hi hello/hi CODE HELLO TO SEE what happens hope we didn't get rid of this baby's babies' wow okay wouldn't want to change"

In [ ]:
_clean_review6(review_sample.loc[24, 'review_text'])

In [ ]:
_clean_review2(review_sample.loc[24, 'review_text'])

In [ ]:
testing = review_sample.loc[24, 'review_text']

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def _filter_characters(text):
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)
    text = re.sub("' ", ' ', text)
    text = re.sub("'s", '', text)
    text = re.sub("'", '', text)
    return text

def _clean_review7(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)
    text = re.sub("' ", ' ', text)
    tokenizer = nltk.RegexpTokenizer('\w+\'?\w*')
    tokens = [token for token in tokenizer.tokenize(text) if token not in new_stops]
    tokens = [token for token in tokens]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
        tokens[i] = re.sub("'s", '', token)
    return tokens



In [ ]:
get_wordnet_pos('babie')

In [ ]:
print(test)
print(_filter_characters(test))

In [ ]:
test1 = test + " joneses'"

In [ ]:
tokenizer = nltk.RegexpTokenizer('\w+\'?\w*')

In [ ]:
tokenizer.tokenize(test1)

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def _filter_characters(text):
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)
    text = re.sub("' ", ' ', text)
    text = re.sub("'s", '', text)
    text = re.sub("'", '', text)
    return text

#For instance like the....cat it's spitting out token 'thecat'
def _clean_review8(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)    
    tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
    tokens = tokenizer.tokenize(text)
    tokens = [token for token in tokenizer.tokenize(text) if token not in new_stops]
    for i, token in enumerate(tokens):
        filtered_token = token.replace("'s", '')
        tokens[i] = wnl.lemmatize(filtered_token, pos= get_wordnet_pos(filtered_token))

    return tokens


In [ ]:
tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
test1 = "testing it's incredible typo's hello-hi hello/hi CODE HELLO TO SEE what happens hope we didn't get rid of this baby's babies' wow okay wouldn't want to change joneses' 1234 hi2 dollars2"

In [ ]:
tokenizer.tokenize(test1)

In [ ]:
tryit = _clean_review8(test1)
tryit.append('crews')
for i, token in enumerate(tryit):
    filtered_token = token.replace("'s", '')
    tryit[i] = wnl.lemmatize(filtered_token, pos= get_wordnet_pos(filtered_token))
print(tryit)

In [ ]:
get_wordnet_pos('changed')

In [ ]:
t.replace("'s", '')

In [ ]:
_clean_review8(testing)

In [ ]:
tokenizer.tokenize(testing)

In [ ]:
testing1 = testing + " babies'" + " baby's"

In [ ]:
tokenizer.tokenize(testing1)

In [ ]:
testing1

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def _filter_characters(text):
    text = re.sub(r"[^A-Za-z\s'\\\/\-]", '', text)
    text = re.sub("' ", ' ', text)
    text = re.sub("'s", '', text)
    text = re.sub("'", '', text)
    return text
###I can add empty token into stopwords.. so empty tokens get removed

def _clean_review9(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
    filtered_tokens = [(re.sub(r"[^A-Za-z\s']", '', token)) for token in tokenizer.tokenize(text)]
    tokens = [token for token in filtered_tokens if token not in new_stops]
    for i, token in enumerate(tokens):
        filtered_token = re.sub("'s", '', token)
        tokens[i] = wnl.lemmatize(filtered_token, pos= get_wordnet_pos(token))
    return tokens

In [ ]:
new_stops

In [ ]:
_clean_review9(testing1)

In [ ]:
new_stops

In [ ]:
text = testing1
text = text.lower()
text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
tokens = [(re.sub(r"[^A-Za-z\s']", '', token)) for token in tokenizer.tokenize(text)]

In [ ]:
tokens

In [ ]:
_clean_review9(testing)

In [ ]:
testing1 = testing + " baby's" + " babies'" + " ladies'"

In [ ]:
_clean_review9(testing1)

In [ ]:
testing1


In [ ]:
testing2 = testing1 + ' hello' + " 'twasn't" + " 'hm"

In [ ]:
testing2

In [ ]:
_clean_review9(testing2)